# Imports

In [3]:
from abc import ABC, abstractmethod
from random import randint
from math import inf
import datetime
from os import path, listdir

# Rubik's cube

In [4]:
class Movement:
  def __init__(self, positions1, positions2, prime, offset):
    self.positions1 = positions1
    self.positions2 = positions2
    self.offset = offset
    self.prime = prime
    if prime:
      self.positions1.reverse()
      self.positions2.reverse()
      self.offset.reverse()

  def apply(self, state):
    for i in range(2):
      aux = state[self.positions1[0] + self.offset[0] * i]
      state[self.positions1[0] + self.offset[0] * i] = state[self.positions1[1] + self.offset[1] * i]
      state[self.positions1[1] + self.offset[1] * i] = state[self.positions1[2] + self.offset[2] * i]
      state[self.positions1[2] + self.offset[2] * i] = state[self.positions1[3] + self.offset[3] * i]
      state[self.positions1[3] + self.offset[3] * i] = aux

    aux = state[self.positions2[0]]
    state[self.positions2[0]] = state[self.positions2[1]]
    state[self.positions2[1]] = state[self.positions2[2]]
    state[self.positions2[2]] = state[self.positions2[3]]
    state[self.positions2[3]] = aux

    return state

class MovementR(Movement):
  def __init__(self, prime):
    super().__init__([1, 13, 21, 5], [9, 8, 10, 11], prime, [2 for i in range(4)]) #ok

  def __str__(self):
    return "R'" if self.prime else "R"

class MovementL(Movement):
  def __init__(self, prime):
    super().__init__([0, 4, 20, 12],[18, 19, 17, 16], prime, [2 for i in range(4)]) #ok

  def __str__(self):
    return "L'" if self.prime else "L"

class MovementU(Movement):
  def __init__(self, prime):
    super().__init__([0, 8, 23, 16],[4, 6, 7, 5], prime, [1, 1, -1, 1]) #ok

  def __str__(self):
    return "U'" if self.prime else "U"

class MovementD(Movement):
  def __init__(self, prime):
    super().__init__([2, 18, 21, 10],[12, 14, 15, 13], prime, [1, 1, -1, 1])#ok

  def __str__(self):
    return "D'" if self.prime else "D"

class MovementF(Movement):
  def __init__(self, prime):
    super().__init__([6, 19, 13, 8],[0, 2, 3, 1], prime, [1, -2, -1, 2])#ok

  def __str__(self):
    return "F'" if self.prime else "F"

class MovementB(Movement):
  def __init__(self, prime):
    super().__init__([4, 9, 15, 18],[23, 21, 20, 22], prime, [1, 2, -1, -2])

  def __str__(self):
    return "B'" if self.prime else "B"

In [5]:
moves = [MovementB(True),  #0
         MovementB(False), #1
         MovementL(True),  #2
         MovementL(False), #3
         MovementD(True),  #4
         MovementD(False), #5
         MovementU(True),  #6
         MovementU(False), #7
         MovementR(True),  #8
         MovementR(False), #9
         MovementF(True),  #10
         MovementF(False), #11
         ]

movementNamesByIndex = {0 : "B'",
                        1 : "B",
                        2: "L'",
                        3: "L",
                        4: "D'",
                        5: "D",
                        6: "U'",
                        7: "U",
                        8: "R'",
                        9: "R",
                        10: "F'",
                        11: "F"}
movementIndexByName = { "B'" : 0,
                        "B" : 1,
                        "L'" : 2,
                        "L" : 3,
                        "D'" : 4,
                        "D" : 5,
                        "U'" : 6,
                        "U" : 7,
                        "R'" : 8,
                        "R" : 9,
                        "F'" : 10,
                        "F" : 11}

In [6]:
class Sequence:
  def __init__(self, movements : list, isRandom=False, size=0):
    global movementIndexByName
    if isRandom:
      self.movements = [randint(0, 11) for i in range(size)]
    else:
      self.movements = movements
      for i in range(len(self.movements)):
        if self.movements[i] in movementIndexByName.keys():
          self.movements[i] = movementIndexByName[self.movements[i]]

  def __str__(self):
    global movementNamesByIndex
    result = ''
    for move in self.movements:
      result = result + movementNamesByIndex[move] + ' '
    return result

In [14]:
class Cube:

  def __init__(self, state = [], solved=False):
    if solved:
      # self.state = ['W' for i in range(4)]       # 0 to 3
      # self.state.extend(['G' for i in range(4)]) # 4 to 7
      # self.state.extend(['O' for i in range(4)]) # 8 to 11
      # self.state.extend(['B' for i in range(4)]) # 12 to 15
      # self.state.extend(['R' for i in range(4)]) # 16 to 19
      # self.state.extend(['Y' for i in range(4)]) # 20 to 23
      self.state = []
      self.state.extend(["W1", "W2", "W3", "W4"])
      self.state.extend(["G1", "G2", "G3", "G4"])
      self.state.extend(["O1", "O2", "O3", "O4"])
      self.state.extend(["B1", "B2", "B3", "B4"])
      self.state.extend(["R1", "R2", "R3", "R4"])
      self.state.extend(["Y1", "Y2", "Y3", "Y4"])
    else:
      self.state = state
    # self.space = "           "
    self.space = "             "

  def movement(self, move : Movement):
    self.state = move.apply(self.state)

  def applySequence(self, movements: Sequence):
    global moves, solvedCube
    fitness = 24
    movesApplied = 0
    lastFitness = 0
    lastImprovementPoint = 0
    for move in movements.movements:
      self.movement(moves[move])
      movesApplied += 1
      newFitness = 24
      for i in range(len(self.state)):
        if self.state[i] == solvedCube.state[i]:
          newFitness -= 1
      if newFitness < fitness:
        lastImprovementPoint = movesApplied
        fitness = newFitness
      if newFitness == 0:
        return (1.0-(1.0/(lastImprovementPoint+1)), movesApplied-1, lastImprovementPoint)
      lastFitness = newFitness
    return (fitness + (1-(1.0/(lastImprovementPoint+1))), movesApplied-1, lastImprovementPoint)

  def shuffle(self, size=50):
    sequence = Sequence([], isRandom=True, size=size)
    print(sequence)
    self.applySequence(sequence)

  def printAloneFace(self, start):
    print(self.space, end = "")
    print(self.state[start:start+2])
    print(self.space, end = "")
    print(self.state[start+2:start+4])

  def printState(self):
    print()
    self.printAloneFace(4)

    print(self.state[16:18], self.state[0:2], self.state[8:10])
    print(self.state[18:20], self.state[2:4], self.state[10:12])


    self.printAloneFace(12)

    self.printAloneFace(20)

    print()

# Genetic Algorithm

In [9]:
class Chromosome:
  def __init__(self, size : int, sequence = None, isRandom = False):
    if isRandom:
      self.sequence = Sequence([], isRandom = True, size = size)
    else:
      self.sequence = sequence

    self.size = size
    self.correct()
    self.fitness, self.movesUntilSolved, self.lastImprovementPoint = self.calculateFitness()

  def correct(self):
    pos = 1
    while pos < len(self.sequence.movements):
      if (self.sequence.movements[pos] % 2 == 0 and self.sequence.movements[pos-1] == self.sequence.movements[pos] + 1) or (self.sequence.movements[pos] % 2 == 1 and self.sequence.movements[pos-1] == self.sequence.movements[pos] - 1) :
        self.sequence.movements.pop(pos)
        pos -= 1
      pos += 1
    while len(self.sequence.movements) < self.size:
      self.sequence.movements.append(randint(0, 11))

  def calculateFitness(self):
    global cube, solvedCube
    newCube = Cube(state=cube.state.copy())
    return newCube.applySequence(self.sequence)

  def __str__(self):
    return str(self.sequence) + " - Fitness = " + str(self.fitness) + f", movesUntilSolved = {self.movesUntilSolved}" + f", lastImprovementPoint = {self.lastImprovementPoint}"

In [10]:
class Crossover(ABC):
  @abstractmethod
  def crossover(parents : list):
    pass

class OnePointCrossover(Crossover):
  @staticmethod
  def crossover(parents : list, point=-1):
    chromosomeSize = parents[0].size
    if point == -1:
      point = randint(1, chromosomeSize - 1)
    moves = []
    moves.extend(parents[0].sequence.movements[i] for i in range(point))
    # moves = parents[0].sequence.movements[:point]
    moves.extend(parents[1].sequence.movements[i] for i in range(point, chromosomeSize))
    # moves.extend(parents[1].sequence.movements[point:])
    sequence = Sequence(moves)
    return Chromosome(chromosomeSize, sequence)

class ImprovementPointCrossover(Crossover):
  @staticmethod
  def crossover(parents : list):
    parents.sort(key = lambda x:x.fitness)
    return OnePointCrossover.crossover(parents, parents[0].lastImprovementPoint if parents[0].lastImprovementPoint < parents[0].size / 2 else -1)

In [11]:
class Mutation(ABC):
  @abstractmethod
  def mutate(chromosome : Chromosome):
    pass

class ExchangeMutation(Mutation):
  @staticmethod
  def mutate(chromosome : Chromosome):
    point1 = randint(0, chromosome.size - 1)
    point2 = randint(0, chromosome.size - 1)
    while point1 == point2:
      point2 = randint(0, chromosome.size - 1)
    newSequence = chromosome.sequence.movements.copy()
    newSequence[point1], newSequence[point2] = newSequence[point2], newSequence[point1]
    return Chromosome(chromosome.size, sequence = Sequence(newSequence))

class ValueMutation(Mutation):
  @staticmethod
  def mutate(chromosome : Chromosome):
    point = randint(0, chromosome.size - 1)
    newSequence = chromosome.sequence.movements.copy()
    newSequence[point] = randint(0, 11)
    return Chromosome(chromosome.size, sequence = Sequence(newSequence))

In [24]:
class GeneticAlgorithm:
  def __init__(self,
               numberOfGenerations : int,
               populationSize : int,
               chromosomeSize : int,
               numberOfCrossovers : int,
               mutationRate : int,
               elitismRate : float,
               maxSimilarityRate : float
               ):
    self.chromosomeSize = chromosomeSize
    self.numberOfGenerations = numberOfGenerations
    self.numberOfCrossovers = numberOfCrossovers
    self.mutationRate = mutationRate
    self.elitismRate = elitismRate
    self.maxSimilarityRate = maxSimilarityRate
    self.populationSize = populationSize
    self.solutionGeneration = -1
    self.bestFitnessPerGenerations = []

  def stopCriteria(self, t, bestSolution):
    # return bestSolution.fitness == 0

    # if self.solutionGeneration == -1:
    #   if bestSolution.fitness < 1:
    #     self.solutionGeneration = t
    #   return False
    # return t >= self.numberOfGenerations + self.solutionGeneration and bestSolution.fitness < 1

    return bestSolution.fitness < 1

  def run(self):
    global cube, solvedCube
    initialTime = datetime.datetime.now()
    self.population = []
    for i in range(self.populationSize):
      self.population.append(Chromosome(size = self.chromosomeSize, isRandom = True))
    t = 0
    self.population.sort(key = lambda x : x.fitness, reverse=False)

    while not self.stopCriteria(t, self.population[0]):
      similarity = 0
      for chromosome in self.population:
        similarity += chromosome.fitness
      similarity /= len(self.population)
      if similarity >= self.maxSimilarityRate * self.population[0].fitness:
        self.population = self.population[:len(self.population)//4]
        while len(self.population) < self.populationSize:
          chromosome = Chromosome(size = self.chromosomeSize, isRandom = True)
          self.population.append(chromosome)

      newPopulation = []
      for crossover in range(self.numberOfCrossovers):
        parent1 = randint(0, self.populationSize - 1)
        parent2 = randint(0, self.populationSize - 1)
        while parent1 == parent2:
          parent2 = randint(0, self.populationSize - 1)
        newPopulation.append(ImprovementPointCrossover.crossover([self.population[parent1], self.population[parent2]]))
        # newPopulation.append(OnePointCrossover.crossover([self.population[parent1], self.population[parent2]]))
        # parents = [self.population[parent1], self.population[parent2]]
        # newPopulation.append(OnePointCrossover.crossover(parents) if randint(0,1) else ImprovementPointCrossover.crossover(parents))
        roulette = randint(0, 99)
        if roulette < self.mutationRate:
          newPopulation[crossover] = ExchangeMutation.mutate(newPopulation[crossover])
          # newPopulation[crossover] = ValueMutation.mutate(newPopulation[crossover])

      for i in range(int(self.elitismRate * self.populationSize)):
        newPopulation.append(self.population[i])
      newPopulation.sort(key = lambda x : x.fitness, reverse=False)
      self.population = newPopulation[:self.populationSize]

      self.bestFitnessPerGenerations.append(self.population[0].fitness)
      t += 1
      if t%1000==0:
        print(f"Generation {t}")
        print(t, self.population[0])
        cubeAfterSequence = Cube(cube.state.copy(), False)
        cubeAfterSequence.applySequence(Sequence(self.population[0].sequence.movements[:self.population[0].movesUntilSolved + 1].copy()))
        cubeAfterSequence.printState()
        print()

    finalTime = datetime.datetime.now()
    time = finalTime - initialTime
    return self.population[0], t, time.total_seconds(), self.bestFitnessPerGenerations

In [ ]:
def main():
  global cube, solvedCube

  cube = Cube(solved=True)
  solvedCube = Cube(solved=True)
  cube.shuffle()

  geneticAlgorithm = GeneticAlgorithm(
                          numberOfGenerations = 250,
                          populationSize = 100,
                          chromosomeSize = 25,
                          numberOfCrossovers = 100,
                          mutationRate = 10,
                          elitismRate = 0.05,
                          maxSimilarityRate = 0.90)

  print(f"Starting algorithm")
  print("Cube state:")
  cube.printState()
  solution, generations, time, bestFitnessPerGeneration = geneticAlgorithm.run()
  print(f"Genetic Algorithm completed")
  print()
  print("Applying solution")
  print(solution)
  print(f"Sequence to apply: {[movementNamesByIndex[i] for i in solution.sequence.movements[:solution.movesUntilSolved + 1].copy()]}")

  # cubeAfterSequence = Cube(cube.state.copy(), False)
  cubeAfterSequence = Cube([a for a in cube.state], False)
  print("Before applying:")
  cubeAfterSequence.printState()
  cubeAfterSequence.applySequence(Sequence(solution.sequence.movements[:solution.movesUntilSolved + 1].copy()))
  print("After applying")
  cubeAfterSequence.printState()

if __name__ == "__main__":
  """run"""
  main()

D' F F L D' L' U' F' D' L U' B' R' D D' U' L' F' R L 
Starting algorithm
Cube state:

             ['B3', 'O2']
             ['O4', 'R4']
['Y1', 'B4'] ['Y2', 'B1'] ['W3', 'Y4']
['G3', 'G1'] ['R1', 'G4'] ['O1', 'W4']
             ['Y3', 'W2']
             ['W1', 'B2']
             ['R2', 'O3']
             ['R3', 'G2']

Geração 1000
1000 U B' L D R D R' F D' L' L' F' U B L' B' D R R L B' B' U B B'  - Fitness = 6.928571428571429, movesUntilSolved = 24, lastImprovementPoint = 13

             ['O2', 'G4']
             ['B2', 'R3']
['G2', 'O3'] ['W4', 'Y1'] ['B3', 'W2']
['G3', 'R4'] ['W3', 'R1'] ['Y3', 'Y2']
             ['B1', 'G1']
             ['W1', 'O4']
             ['R2', 'B4']
             ['Y4', 'O1']


Geração 2000
2000 U B' L D R D R' F D' L' L' F' U B F' L' B B D' F U' F' L' F' U'  - Fitness = 6.928571428571429, movesUntilSolved = 24, lastImprovementPoint = 13

             ['R1', 'O1']
             ['R4', 'B2']
['Y3', 'B1'] ['W3', 'O3'] ['W4', 'G4']
['O2', 'B3'] ['Y1', 'W1'] [